# **Deepfake Detection Project**

## **1. About the Project**
The **Deepfake Detection Project** is designed to distinguish between **real** and **manipulated** facial images using deep learning. The project employs:

- **MobileNetV2-based classifier** for binary classification (Real/Fake).
- **Autoencoder** for anomaly detection.
- **Grad-CAM** for explainability (visual heatmaps).
- **Face landmarks detection** and **blur analysis** for enhanced detection.
- **Gradio UI** for real-time deepfake detection on images and videos.

---

## **2. Workflow Overview**
The project consists of the following stages:

### **📌 Data Preparation and EDA**
- Dataset: `https://www.kaggle.com/competitions/deepfake-detection-challenge`
- Exploratory Data Analysis (EDA) includes:
  - Dataset distribution visualization
  - Sample image previews
  - Metadata exploration

### **📌 Model Training**
1. **Autoencoder Training**
   - Learns latent representations of real images.
   - Uses **Mean Squared Error (MSE)** loss.
   - Architecture: CNN-based encoder-decoder.

2. **Deepfake Classifier Training**
   - **MobileNetV2** with a modified final classification layer.
   - Uses **Cross-Entropy Loss**.
   - Optimized with **Adam optimizer**.

### **📌 Feature Extraction**
- **Grad-CAM**: Highlights important image regions.
- **Face Landmarks**: Uses dlib for key feature extraction.
- **Blur Analysis**: Uses Laplacian variance to detect artifacts.
- **Anomaly Detection**: Autoencoder reconstruction errors.

### **📌 Deployment via Gradio**
- **Image-Based Detection**
- **Video-Based Detection**
- **Tabbed Interface for User Interaction**

---





In [ ]:
!pip install torch torchvision gradio dlib opencv-python matplotlib numpy captum

In [ ]:
# train_deepfake_detector.py

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((96, 96)),  # Resize images to 96x96
    transforms.ToTensor(),        # Convert image to tensor
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1,1] range
])

# Load dataset
train_path = "/content/drive/MyDrive/G15/archive/real_and_fake_face/train"
test_path = "/content/drive/MyDrive/G15/archive/real_and_fake_face/test"

train_dataset = ImageFolder(root=train_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load MobileNetV2 model
model = mobilenet_v2(pretrained=True)

# Modify classifier to match 2-class classification (Real vs Fake)
model.classifier[1] = nn.Linear(model.last_channel, 2)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

# Save trained model
torch.save(model.state_dict(), "deepfake_detector.pth")
print("Model saved as 'deepfake_detector.pth'")


In [10]:
# train_autoencoder.py

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import os

# Define the Autoencoder model for unsupervised anomaly detection
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder: reduces image dimensionality
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),  # 16x48x48
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # 32x24x24
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # 64x12x12
            nn.ReLU()
        )
        # Decoder: reconstructs original image
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),  # 32x24x24
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),  # 16x48x48
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),   # 1x96x96
            nn.Sigmoid()  # Output in range [0, 1]
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((96, 96)),
    transforms.ToTensor()
])

# Load dataset
dataset_path = "/content/drive/MyDrive/G15/archive/real_and_fake_face"
dataset = ImageFolder(root=dataset_path, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model, loss, optimizer
autoencoder = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for imgs, _ in loader:
        imgs = imgs.to(device)
        optimizer.zero_grad()
        outputs = autoencoder(imgs)
        loss = criterion(outputs, imgs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(loader):.4f}")

# Save the trained autoencoder model
torch.save(autoencoder.state_dict(), "autoencoder.pth")
print("Autoencoder model saved as 'autoencoder.pth'")


Epoch [1/10], Loss: 0.0451
Epoch [2/10], Loss: 0.0108
Epoch [3/10], Loss: 0.0057
Epoch [4/10], Loss: 0.0046
Epoch [5/10], Loss: 0.0040
Epoch [6/10], Loss: 0.0037
Epoch [7/10], Loss: 0.0034
Epoch [8/10], Loss: 0.0034
Epoch [9/10], Loss: 0.0031
Epoch [10/10], Loss: 0.0031
Autoencoder model saved as 'autoencoder.pth'


In [18]:
# deepfake_detector.py

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
import numpy as np
import cv2
import gradio as gr
import dlib
from PIL import Image
from captum.attr import GuidedGradCam

#######################################
# Load MobileNetV2-based Deepfake Detector
#######################################
model = mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 2)
model.load_state_dict(torch.load('deepfake_detector.pth', map_location=torch.device('cpu')))
model.eval()

# Grad-CAM layer
target_layer = model.features[-1]

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#######################################
# Image Preprocessing Function
#######################################
def preprocess_image(image):
    transform_fn = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((96, 96)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    return transform_fn(image).unsqueeze(0)

#######################################
# Grad-CAM for Fake Region Detection
#######################################
def grad_cam(image):
    image_tensor = preprocess_image(image)
    guided_gc = GuidedGradCam(model, target_layer)
    cam = guided_gc.attribute(image_tensor, target=1).detach().numpy()[0]
    cam = np.transpose(cam, (1, 2, 0))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8) * 255
    return cam.astype(np.uint8)

#######################################
# Blur Detection (Deepfake Artifacts)
#######################################
def detect_blur(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return "Blurry" if variance < 100 else "Sharp"

#######################################
# Multi-Face Deepfake Detection
#######################################
def detect_landmarks(image):
    img_copy = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    results = []
    for face in faces:
        landmarks = predictor(gray, face)
        for i in range(68):
            x, y = landmarks.part(i).x, landmarks.part(i).y
            cv2.circle(img_copy, (x, y), 1, (0, 255, 0), -1)
        results.append((face, img_copy))
    return results if results else [(None, img_copy)]

#######################################
# Predict Function with Confidence Score
#######################################
def predict(image):
    processed = preprocess_image(image)
    with torch.no_grad():
        output = model(processed)
        probabilities = torch.nn.functional.softmax(output, dim=1)
        confidence = probabilities[0][1].item() * 100
    result = f"Real ({confidence:.2f}%)" if confidence > 50 else f"Fake ({100-confidence:.2f}%)"

    gradcam_output = grad_cam(image)
    landmarks_output = detect_landmarks(image.copy())[0][1]
    blur_result = detect_blur(image)

    return result, gradcam_output, landmarks_output, blur_result

#######################################
# Video Frame-by-Frame Analysis
#######################################
def analyze_video(video):
    cap = cv2.VideoCapture(video)
    frames = []
    results = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        result, _, _, _ = predict(frame)
        frames.append(frame)
        results.append(result)
    cap.release()
    return results

#######################################
# Gradio Interfaces
#######################################
image_ui = gr.Interface(
    fn=predict,
    inputs=[gr.Image(type='numpy', label="Input Image")],
    outputs=[
        gr.Label(label="Classification"),
        gr.Image(type='numpy', label="Grad-CAM"),
        gr.Image(type='numpy', label="Face Landmarks"),
        gr.Label(label="Blur Detection")
    ],
    title="Enhanced Deepfake Image Detector",
    description="Upload an image to check if it's real or fake with confidence score and visual analysis."
)

video_ui = gr.Interface(
    fn=analyze_video,
    inputs=gr.Video(label="Input Video")
,
    outputs=gr.Textbox(label="Frame-by-Frame Deepfake Analysis"),
    title="Deepfake Video Detector",
    description="Upload a video to analyze each frame for deepfakes."
)

dashboard = gr.TabbedInterface([image_ui, video_ui], ["Image Detection", "Video Analysis"])
dashboard.launch()


<ipython-input-18-3c39ec891688>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('deepfake_detector.pth', map_location=torch.device('cpu'))

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e90a2489dd843ed1c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
